In [5]:
from torchmetrics.image import StructuralSimilarityIndexMeasure
import numpy as np
import torch

In [6]:

def compute_residual_images(img_set1, img_set2):
    #convert to float   
    img_set1 = img_set1.astype(np.float32)
    img_set2 = img_set2.astype(np.float32)
    # Ensure the shapes match
    assert img_set1.shape == img_set2.shape, "Image sets must have the same dimensions"
    
    # Compute the residuals
    residuals = np.abs(img_set1 - img_set2)
    return residuals

def calc_ssim_batched(img_set1, img_set2, batch_size=128):
    # Initialize the SSIM metric
    ssim = StructuralSimilarityIndexMeasure(data_range=255, reduction="none")
    
    # Ensure input tensors are of float32 type
    img_set1 = torch.tensor(img_set1, dtype=torch.float32)
    img_set2 = torch.tensor(img_set2, dtype=torch.float32)

    # Calculate SSIM in batches
    ssim_vals = []
    for i in range(0, len(img_set1), batch_size):
        batch_img1 = img_set1[i:i + batch_size]
        batch_img2 = img_set2[i:i + batch_size]
        
        # Compute SSIM for the batch and store the result
        ssim_batch = ssim(batch_img1, batch_img2)
        ssim_vals.append(ssim_batch.cpu().numpy())
    # Concatenate all batch results into a single array
    return np.concatenate(ssim_vals)
def calculate_composite_score(img_set1, img_set2, alpha=10, batch_size=128):
    ssim = calc_ssim_batched(img_set1, img_set2, batch_size=batch_size)
    ssim_rescaled = (ssim + 1) / 2
    residuals = compute_residual_images(img_set1, img_set2)
    MAE = residuals/255
    MAE = np.mean(MAE, axis=(1, 2, 3))
    composite_score = (ssim_rescaled + alpha*(1-MAE))/(1+alpha)
    return np.mean(composite_score), composite_score


In [9]:
submission = np.load("1739_7842_3460.npy")
score, _ = calculate_composite_score(submission, np.load('data/images_test.npy'))
print(score)

0.80429745
